In [ ]:
# Alphabetical Order from Github

In [ ]:
# bing_maps_images_min_max.py #
import requests
import os
import json  # Import the json module to work with JSON files

# Define the base API URL
base_url = "https://dev.virtualearth.net/REST/v1/Imagery/Map/Aerial/"

# Define the query parameters that are common to all requests
common_params = {
    "mapSize": "500,500",
    "mapLayer": "Basemap,Buildings",
    "key": "Ain7kUv28hvUkTkX5QfhVU-J_rqqtZMk7lGZNjh_e0ivB3wxcJsR3tAHJVAr8ZdC",
}

# Load the dataset of IDs and their corresponding coordinates from min_max_coords.json
with open("coordinates.json", "r") as json_file:
    coordinates_data = json.load(json_file)

# Create a directory to store the images
output_dir = "bing_maps_images_min_max"
os.makedirs(output_dir, exist_ok=True)

# Iterate through the dataset and make API requests
for id, coords in coordinates_data.items():
    # Define unique parameters for each request
    params = {
        **common_params,
        "mapArea": f"{coords['min_lat']},{coords['min_lon']},{coords['max_lat']},{coords['max_lon']}",
    }

    # Make the API request
    response = requests.get(base_url, params=params)

    # Check for a successful response
    if response.status_code == 200:
        # Save the image data to a file
        image_file = os.path.join(output_dir, f"{id}_image.jpg")
        with open(image_file, "wb") as file:
            file.write(response.content)
        print(f"Image for ID {id} saved as {image_file}")
    else:
        # Print an error message if the request was not successful
        print(f"HTTP request for ID {id} failed with status code: {response.status_code}")

In [ ]:
# bing_maps_query_center_point.py #
import requests
import os
import json  # Import the json module to work with JSON files

# Define the base API URL
def api_url(lat, long):
    base_url = "https://dev.virtualearth.net/REST/v1/Imagery/Map/Aerial/"
    # Define the query parameters that are common to all requests
    mapSize = "500,500"
    mapLayer = "Basemap,Buildings"
    key = "Ain7kUv28hvUkTkX5QfhVU-J_rqqtZMk7lGZNjh_e0ivB3wxcJsR3tAHJVAr8ZdC"
    zoomLevel = "19"

    url = f"{base_url}{lat},{long}{zoomLevel}?mapSize={mapSize}&mapLayer={mapLayer}&key={key}&dc="

    response = requests.get(url)

    # Check for a successful response
    if response.status_code == 200:
        # Save the image data to a file
        image_file = os.path.join(output_dir, f"{id}_image.jpg")
        with open(image_file, "wb") as file:
            file.write(response.content)
        print(f"Image for ID {id} saved as {image_file}")
    else:
        # Print an error message if the request was not successful
        print(f"HTTP request for ID {id} failed with status code: {response.status_code}")

# Load the dataset of IDs and their corresponding coordinates from min_max_coords.json
with open("coordinates.json", "r") as json_file:
    coordinates_data = json.load(json_file)

# Create a directory to store the images
output_dir = "bing_maps_images_center_point"
os.makedirs(output_dir, exist_ok=True)

# Iterate through the dataset and make API requests
for id, coords in coordinates_data.items():
    # Define unique parameters for each request
    lat = coords["center_point_lat"]
    long = coords["center_point_lon"]

    response_data = api_url(lat, long)

In [ ]:
# color_change.py #
import os
from PIL import Image

def change_color(image_path, target_color, replacement_color):
    # Open the image
    img = Image.open(image_path)

    # Get the image data
    pixels = img.load()

    # Iterate through each pixel
    for i in range(img.width):
        for j in range(img.height):
            # Get the current pixel color
            current_color = pixels[i, j]

            # Check if the current pixel color is NOT equal to the target color
            if current_color != target_color:
                # Replace the color with the specified replacement color
                pixels[i, j] = replacement_color

    # Save the modified image
    img.save(image_path.replace('bing_maps_images_mask', 'bing_maps_images_mask/modified_images'))

if __name__ == "__main__":
    # Folder containing the images
    folder_path = "bing_maps_images_mask"

    # Specify the target ARGB color (pink in this case)
    target_color = (255, 80, 100)  # Pink

    # Specify the replacement ARGB color (black in this case)
    replacement_color = (0, 0, 0)  # Black

    # Create a folder for modified images if it doesn't exist
    output_folder = os.path.join(folder_path, 'modified_images')
    os.makedirs(output_folder, exist_ok=True)

    # Iterate through each file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            # Get the path of each image file
            image_path = os.path.join(folder_path, filename)

            # Call the function for each image
            change_color(image_path, target_color, replacement_color)

In [ ]:
# coordinates.py #
import requests
import json

# Overpass API URL
overpass_url = "http://overpass-api.de/api/interpreter"

# Your Overpass query
overpass_query = """
[out:json][timeout:100];
// gather results
(
  // query part for: “parking=surface”
  node["parking"="surface"](39.7042,-86.3994,39.9569,-86.0172);
  way["parking"="surface"](39.7042,-86.3994,39.9569,-86.0172);
  relation["parking"="surface"](39.7042,-86.3994,39.9569,-86.0172);
);
// print results
out body;
>;
out skel qt;
"""

# Send the request
overpass_response = requests.get(overpass_url, params={'data': overpass_query})

# Check for HTTP errors
overpass_response.raise_for_status()

# Parse the JSON response
data = overpass_response.json()

# Print the results or process them as you need
print(json.dumps(data, indent=4))

# Save the data to a file
with open('data.json', 'w') as file:
    json.dump(data, file, indent=4)


# Method to process the Overpass API Query
def process_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)

    # Create a dictionary to map node IDs to their lat/lon
    node_dict = {}
    for element in data['elements']:
        if element['type'] == 'node':
            node_dict[element['id']] = (element['lat'], element['lon'])

    output_data = {}
    for element in data['elements']:
        if element['type'] == 'way':
            way_id = str(element['id'])
            node_ids = element['nodes']

            # Initialize min and max values
            min_lat = min_lon = float('inf')
            max_lat = max_lon = float('-inf')
            total_lat = total_lon = 0
            coordinate_list = []

            for node_id in node_ids:
                if node_id in node_dict:
                    lat, lon = node_dict[node_id]
                    min_lat = min(min_lat, lat)
                    max_lat = max(max_lat, lat)
                    min_lon = min(min_lon, lon)
                    max_lon = max(max_lon, lon)
                    total_lat += lat
                    total_lon += lon
                    coordinate_list.append(f"{lat},{lon}")

            center_lat = total_lat / len(node_ids)
            center_lon = total_lon / len(node_ids)
            coordinates = "_".join(coordinate_list)

            output_data[way_id] = {
                "min_lat": min_lat,
                "min_lon": min_lon,
                "max_lat": max_lat,
                "max_lon": max_lon,
                "center_point_lat": center_lat,
                "center_point_lon": center_lon,
                "coordinates": coordinates
            }

    return output_data

# Replace 'path_to_your_data.json' with the actual file path
processed_data = process_json('data.json')

# To print the processed data
print(json.dumps(processed_data, indent=4))

# To save the processed data to a new file
with open('coordinates.json', 'w') as file:
    json.dump(processed_data, file, indent=4)

In [ ]:
# get_testing_data #
import requests
import math
import os

# Ensure the folder exists
save_dir = 'D:/OSMTestingImages'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Define your bounding box and ground resolution
min_lat = 39.7416667
min_long = -86.1833333
max_lat = 39.7853
max_long = -86.1340
ground_resolution = 0.2986  # meters/pixel; Change this image to adjust zoom level

# Convert ground resolution to delta in degrees for latitude and longitude
delta_lat = ground_resolution / 111111  # degrees per meter (approximate)
delta_long = delta_lat / math.cos(math.radians(min_lat))

# Calculate the number of squares within the bounding box
num_lat = int((max_lat - min_lat) / delta_lat) + 1
num_long = int((max_long - min_long) / delta_long) + 1

# API parameters
base_url = "https://dev.virtualearth.net/REST/v1/Imagery/Map/Aerial/"
constant_params = {
    "mapSize": "500,500",
    "mapLayer": "Basemap,Buildings",
    "key": "Ain7kUv28hvUkTkX5QfhVU-J_rqqtZMk7lGZNjh_e0ivB3wxcJsR3tAHJVAr8ZdC"  # Bing Maps API key
}

# Function to retrieve and save the image
def get_image(bottom_left_lat, bottom_left_long, top_right_lat, top_right_long):
    # Construct the map area string
    map_area_value = f"{bottom_left_lat},{bottom_left_long},{top_right_lat},{top_right_long}"

    # Construct the API URL
    params = {
        **constant_params,
        "mapArea": map_area_value
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        # Construct the file path
        image_filename = os.path.join(save_dir, f"image_{bottom_left_lat}_{bottom_left_long}_{top_right_lat}_{top_right_long}.jpeg")
        # Save the image
        with open(image_filename, 'wb') as image_file:
            image_file.write(response.content)
        print(f"Downloaded {image_filename}")
    else:
        print(f"Failed to get image at {bottom_left_lat}, {bottom_left_long}, {top_right_lat}, {top_right_long}: {response.status_code}")

# Iterate over each square within the bounding box

for lat_step in range(5):
    for long_step in range(5):
        # Calculate the latitude and longitude for the bottom left corner of the square
        bottom_left_lat = min_lat + (lat_step * delta_lat)
        bottom_left_long = min_long + (long_step * delta_long)

        # Calculate the latitude and longitude for the top right corner of the square
        top_right_lat = bottom_left_lat + delta_lat
        top_right_long = bottom_left_long + delta_long

        # Ensure that we do not exceed the max bounds
        top_right_lat = min(top_right_lat, max_lat)
        top_right_long = min(top_right_long, max_long)

        # Retrieve and save the image for the current square
        get_image(bottom_left_lat, bottom_left_long, top_right_lat, top_right_long)

In [ ]:
# img_to_tensor.py #
# Import necessary libraries
import torch
from PIL import Image
import torchvision.transforms as transforms

# Read a PIL image
image = Image.open('IMAGE')

# Define a transform to convert PIL
# image to a Torch tensor
transform = transforms.Compose([
    transforms.PILToTensor()
])

# transform = transforms.PILToTensor()
# Convert the PIL image to Torch tensor
img_tensor = transform(image)

# print the converted Torch tensor
print(img_tensor)

In [ ]:
# json_to_osm.py #
import json
import xml.etree.ElementTree as ET
from xml.dom import minidom

# Load JSON data from your file
with open('data.json', 'r') as json_file:
    data = json.load(json_file)

# Create the root OSM element
osm = ET.Element('osm', version=str(data['version']), generator=data['generator'])

# Loop through the 'elements' list and convert them to OSM XML elements
for element in data['elements']:
    if element['type'] == 'node':
        node = ET.SubElement(osm, 'node', id=str(element['id']), lat=str(element['lat']), lon=str(element['lon']))
    # You can handle other element types (e.g., 'way', 'relation') here if needed

# Create a prettified XML string
xml_str = minidom.parseString(ET.tostring(osm)).toprettyxml(indent="  ")

# Write the XML string to a file
with open('output.osm', 'w') as osm_file:
    osm_file.write(xml_str)

print("Conversion completed. Output saved as 'output.osm'.")

In [ ]:
# mask_creation #
import requests
import json
import os

# Function to generate the URL based on coordinates
def generate_url(coords):
    base_url = "https://dev.virtualearth.net/REST/v1/Imagery/Map/Aerial/"

    # Create a folder to store the images if it doesn't exist
    folder_name = "bing_maps_images_mask"
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    for way_id, way_data in coords.items():
        center_lat = way_data["center_point_lat"]
        center_lon = way_data["center_point_lon"]
        zoom = 19
        map_size = "500,500"
        coordinates = way_data["coordinates"]

        url = f"{base_url}{center_lat},{center_lon}/{zoom}"
        map_layer = "Basemap,Buildings"
        key = "Ain7kUv28hvUkTkX5QfhVU-J_rqqtZMk7lGZNjh_e0ivB3wxcJsR3tAHJVAr8ZdC"
        map_size_str = f"&mapSize={map_size}"
        dc = f"p,FFFF5064,FFFF5064,2;{coordinates}"

        full_url = f"{url}?mapLayer={map_layer}&key={key}{map_size_str}&dc={dc}"
        response = requests.get(full_url)

        if response.status_code == 200:
            # Save the image to the folder
            with open(f"{folder_name}/image_{way_id}.jpg", "wb") as file:
                file.write(response.content)
            print(f"Image for way {way_id} saved")
        else:
            print(f"Error querying for way {way_id}: {response.status_code}")

# Load coordinates from min_max_coords.json
with open("min_max_coords.json", "r") as coords_file:
    coordinates_data = json.load(coords_file)

# Generate URLs based on coordinates and save the images
generate_url(coordinates_data)

In [ ]:
# raster_to_vector #
import matplotlib.pyplot as plt
from skimage import io, color, measure
import numpy as np

# Load the raster image of the parking lot (masked_image.jpg)
image_path = 'image_1051403817.jpg'
image_array = io.imread(image_path)
image_gray = color.rgb2gray(image_array)

# Create a mask to identify the black parts (assuming black is the area to trace)
mask = (image_gray < 0.1)  # Modify the threshold as needed

# Apply the mask to the grayscale image
masked_image = np.where(mask, 1, 0)

# Invert the mask to trace the edges
inverted_mask = np.invert(mask)

# Contour detection using skimage's `find_contours` function on the inverted mask
contours = measure.find_contours(inverted_mask, 0.8)

# Find the contour with the largest area
largest_contour = max(contours, key=len)

# Function to find points with higher curvature (edge points)
def find_edge_points(contour, num_points):
    edge_indices = np.linspace(0, len(contour) - 1, num_points, dtype=int)
    return contour[edge_indices].astype(int)

# Function to find intersection points of a line with a polygon
def find_intersection_points(poly, line):
    intersections = []
    for i in range(len(poly) - 1):
        for j in range(len(line) - 1):
            x1, y1 = poly[i]
            x2, y2 = poly[i + 1]
            x3, y3 = line[j]
            x4, y4 = line[j + 1]
            denominator = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
            if denominator != 0:
                px = ((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / denominator
                py = ((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / denominator
                if min(x1, x2) <= px <= max(x1, x2) and min(y1, y2) <= py <= max(y1, y2):
                    intersections.append((px, py))
    return np.array(intersections)

# Create a figure and axis for plotting
fig, ax = plt.subplots()

# Display the grayscale image with inverted mask
ax.imshow(image_gray, cmap='gray')

# Extract 10 edge points from the largest contour
edge_points = find_edge_points(largest_contour, 10)

# Plot the edges of the largest contour
ax.plot(largest_contour[:, 1], largest_contour[:, 0], color='blue')

# Plot the sampled edge points (dots) only on the edge of the shape
ax.scatter(edge_points[:, 1], edge_points[:, 0], color='red', s=10)

# Create a straight line from the first to the last edge point
start_point = edge_points[0]
end_point = edge_points[-1]
straight_line = np.array([start_point, end_point])

# Find intersection points between the straight line and the contour
intersections = find_intersection_points(largest_contour, straight_line)
ax.scatter(intersections[:, 1], intersections[:, 0], color='red', s=30, marker='x')

# Print the coordinates of the intersection points
print("Coordinates of the intersection points:")
for point in intersections:
    print(point)

# Show the plot
plt.show()

In [ ]:
# request_osm_api.py #
import requests

# Define the API endpoint
osm_api_url = 'https://api.openstreetmap.org/api/0.6/map?bbox=-180,-90,180,90'

# Set up request headers (e.g., Authorization header for authenticated requests)
headers = {
    'Content-Type': 'application/xml',
    'Authorization': 'Bearer YOUR_ACCESS_TOKEN'  # Only for authenticated requests
}

# Send a GET request to the API
response = requests.get(osm_api_url, headers=headers)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the XML response (response.text) and process the data
    osm_data = response.text
    # You can parse the XML data as needed
    print(osm_data)
else:
    print(f'Error: {response.status_code} - {response.text}')

In [ ]:
# test.py #
import os

def remove_image_strings(directory):
    for filename in os.listdir(directory):
        if 'image_' in filename:
            new_filename = filename.replace('image_', '')
            os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))
        elif '_image' in filename:
            new_filename = filename.replace('_image', '')
            os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))

# Replace 'path/to/your/directory' with the directory path containing your files
directory_path = 'bing_maps_images_mask/bing_maps_images_completed_mask'
remove_image_strings(directory_path)

In [ ]:
# tilesystem.py #
import math
import os
import requests

class TileSystem:
    EarthRadius = 6378137
    MinLatitude = -85.05112878
    MaxLatitude = 85.05112878
    MinLongitude = -180
    MaxLongitude = 180

    @staticmethod
    def clip(n, minValue, maxValue):
        """
        Clips a number to the specified minimum and maximum values.

        Parameters:
        n (float): The number to clip.
        minValue (float): Minimum allowable value.
        maxValue (float): Maximum allowable value.

        Returns:
        float: The clipped value.
        """
        return max(min(n, maxValue), minValue)

    @staticmethod
    def map_size(level_of_detail):
        """
        Determines the map width and height (in pixels) at a specified level of detail.

        Parameters:
        level_of_detail (int): Level of detail, from 1 (lowest detail) to 23 (highest detail).

        Returns:
        int: The map width and height in pixels.
        """
        return 256 << level_of_detail

    @staticmethod
    def ground_resolution(latitude, level_of_detail):
        """
        Determines the ground resolution (in meters per pixel) at a specified latitude and level of detail.

        Parameters:
        latitude (float): Latitude (in degrees) at which to measure the ground resolution.
        level_of_detail (int): Level of detail, from 1 (lowest detail) to 23 (highest detail).

        Returns:
        float: The ground resolution, in meters per pixel.
        """
        latitude = TileSystem.clip(latitude, TileSystem.MinLatitude, TileSystem.MaxLatitude)
        return math.cos(latitude * math.pi / 180) * 2 * math.pi * TileSystem.EarthRadius / TileSystem.map_size(level_of_detail)

    @staticmethod
    def map_scale(latitude, level_of_detail, screen_dpi):
        """
        Determines the map scale at a specified latitude, level of detail, and screen resolution.

        Parameters:
        latitude (float): Latitude (in degrees) at which to measure the map scale.
        level_of_detail (int): Level of detail, from 1 (lowest detail) to 23 (highest detail).
        screen_dpi (int): Resolution of the screen, in dots per inch.

        Returns:
        float: The map scale, expressed as the denominator N of the ratio 1 : N.
        """
        return TileSystem.ground_resolution(latitude, level_of_detail) * screen_dpi / 0.0254

    @staticmethod
    def lat_long_to_pixel_xy(latitude, longitude, level_of_detail):
        """
        Converts a point from latitude/longitude WGS-84 coordinates (in degrees) into pixel XY coordinates at a specified level of detail.

        Parameters:
        latitude (float): Latitude of the point, in degrees.
        longitude (float): Longitude of the point, in degrees.
        level_of_detail (int): Level of detail, from 1 (lowest detail) to 23 (highest detail).

        Returns:
        tuple: A tuple containing the X and Y coordinates in pixels.
        """
        latitude = TileSystem.clip(latitude, TileSystem.MinLatitude, TileSystem.MaxLatitude)
        longitude = TileSystem.clip(longitude, TileSystem.MinLongitude, TileSystem.MaxLongitude)

        x = (longitude + 180) / 360
        sin_latitude = math.sin(latitude * math.pi / 180)
        y = 0.5 - math.log((1 + sin_latitude) / (1 - sin_latitude)) / (4 * math.pi)

        map_size = TileSystem.map_size(level_of_detail)
        pixel_x = int(TileSystem.clip(x * map_size + 0.5, 0, map_size - 1))
        pixel_y = int(TileSystem.clip(y * map_size + 0.5, 0, map_size - 1))
        return pixel_x, pixel_y

    @staticmethod
    def pixel_xy_to_lat_long(pixel_x, pixel_y, level_of_detail):
        """
        Converts a pixel from pixel XY coordinates at a specified level of detail into latitude/longitude WGS-84 coordinates (in degrees).

        Parameters:
        pixel_x (int): X coordinate of the point, in pixels.
        pixel_y (int): Y coordinate of the point, in pixels.
        level_of_detail (int): Level of detail, from 1 (lowest detail) to 23 (highest detail).

        Returns:
        tuple: A tuple containing the latitude and longitude in degrees.
        """
        map_size = TileSystem.map_size(level_of_detail)
        x = (TileSystem.clip(pixel_x, 0, map_size - 1) / map_size) - 0.5
        y = 0.5 - (TileSystem.clip(pixel_y, 0, map_size - 1) / map_size)

        latitude = 90 - 360 * math.atan(math.exp(-y * 2 * math.pi)) / math.pi
        longitude = 360 * x
        return latitude, longitude

    @staticmethod
    def pixel_xy_to_tile_xy(pixel_x, pixel_y):
        """
        Converts pixel XY coordinates into tile XY coordinates of the tile containing the specified pixel.

        Parameters:
        pixel_x (int): Pixel X coordinate.
        pixel_y (int): Pixel Y coordinate.

        Returns:
        tuple: A tuple containing the tile X and Y coordinates.
        """
        tile_x = pixel_x // 256
        tile_y = pixel_y // 256
        return tile_x, tile_y

    @staticmethod
    def tile_xy_to_pixel_xy(tile_x, tile_y):
        """
        Converts tile XY coordinates into pixel XY coordinates of the upper-left pixel of the specified tile.

        Parameters:
        tile_x (int): Tile X coordinate.
        tile_y (int): Tile Y coordinate.

        Returns:
        tuple: A tuple containing the pixel X and Y coordinates.
        """
        pixel_x = tile_x * 256
        pixel_y = tile_y * 256
        return pixel_x, pixel_y

    @staticmethod
    def tile_xy_to_quad_key(tile_x, tile_y, level_of_detail):
        """
        Converts tile XY coordinates into a QuadKey at a specified level of detail.

        Parameters:
        tile_x (int): Tile X coordinate.
        tile_y (int): Tile Y coordinate.
        level_of_detail (int): Level of detail, from 1 (lowest detail) to 23 (highest detail).

        Returns:
        str: A string containing the QuadKey.
        """
        quad_key = []
        for i in range(level_of_detail, 0, -1):
            digit = '0'
            mask = 1 << (i - 1)
            if (tile_x & mask) != 0:
                digit = chr(ord(digit) + 1)
            if (tile_y & mask) != 0:
                digit = chr(ord(digit) + 2)
            quad_key.append(digit)
        return ''.join(quad_key)

    @staticmethod
    def quad_key_to_tile_xy(quad_key):
        """
        Converts a QuadKey into tile XY coordinates.

        Parameters:
        quad_key (str): QuadKey of the tile.

        Returns:
        tuple: A tuple containing the tile X and Y coordinates and the level of detail.
        """
        tile_x = tile_y = 0
        level_of_detail = len(quad_key)
        for i in range(level_of_detail):
            mask = 1 << (level_of_detail - i - 1)
            if quad_key[i] == '0':
                continue
            elif quad_key[i] == '1':
                tile_x |= mask
            elif quad_key[i] == '2':
                tile_y |= mask
            elif quad_key[i] == '3':
                tile_x |= mask
                tile_y |= mask
            else:
                raise ValueError("Invalid QuadKey digit sequence.")
        return tile_x, tile_y, level_of_detail

import requests
import os

def get_quadkeys_for_region(bottom_left_lat, bottom_left_lon, top_right_lat, top_right_lon, zoom_level):
    bottom_left_pixel_x, bottom_left_pixel_y = TileSystem.lat_long_to_pixel_xy(bottom_left_lat, bottom_left_lon, zoom_level)
    top_right_pixel_x, top_right_pixel_y = TileSystem.lat_long_to_pixel_xy(top_right_lat, top_right_lon, zoom_level)

    bottom_left_tile_x, bottom_left_tile_y = TileSystem.pixel_xy_to_tile_xy(bottom_left_pixel_x, bottom_left_pixel_y)
    top_right_tile_x, top_right_tile_y = TileSystem.pixel_xy_to_tile_xy(top_right_pixel_x, top_right_pixel_y)

    quadkeys = []
    for tile_x in range(bottom_left_tile_x, top_right_tile_x + 1):
        for tile_y in range(top_right_tile_y, bottom_left_tile_y + 1):
            quadkey = TileSystem.tile_xy_to_quad_key(tile_x, tile_y, zoom_level)
            quadkeys.append(quadkey)

    return quadkeys

def save_images_for_quadkeys(quadkeys, directory, common_params):
    if not os.path.exists(directory):
        os.makedirs(directory)

    for quadkey in quadkeys:
        tile_x, tile_y, level_of_detail = TileSystem.quad_key_to_tile_xy(quadkey)
        pixel_x, pixel_y = TileSystem.tile_xy_to_pixel_xy(tile_x, tile_y)
        lat_top_left, lon_top_left = TileSystem.pixel_xy_to_lat_long(pixel_x, pixel_y, level_of_detail)
        lat_bottom_right, lon_bottom_right = TileSystem.pixel_xy_to_lat_long(pixel_x + 255, pixel_y + 255, level_of_detail)

        image_title = f"map_{lat_bottom_right}_{lon_top_left}_{lat_top_left}_{lon_bottom_right}.jpg"
        file_path = os.path.join(directory, image_title)

        params = {
            **common_params,
            "mapArea": f"{lat_bottom_right},{lon_top_left},{lat_top_left},{lon_bottom_right}",
            "format": "jpeg"
        }
        base_url = "https://dev.virtualearth.net/REST/v1/Imagery/Map/Aerial/"

        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            with open(file_path, "wb") as file:
                file.write(response.content)
        else:
            print(f"Failed to get image at {lat_bottom_right}, {lon_top_left}, {lat_top_left}, {lon_bottom_right}: {response.status_code}")

# Example usage
bottom_left_lat = 39.7416667  # Replace with actual latitude
bottom_left_lon = -86.1833333  # Replace with actual longitude
top_right_lat = 39.7853 # Replace with actual latitude
top_right_lon = -86.1340  # Replace with actual longitude
zoom_level = 19
quadkeys = get_quadkeys_for_region(bottom_left_lat, bottom_left_lon, top_right_lat, top_right_lon, zoom_level)

common_params = {
    "mapSize": "500,500",
    "mapLayer": "Basemap,Buildings",
    "key": "Ain7kUv28hvUkTkX5QfhVU-J_rqqtZMk7lGZNjh_e0ivB3wxcJsR3tAHJVAr8ZdC"  # Replace with your Bing Maps API Key
}
directory = "D:/OSMTestingImagesZL19"
save_images_for_quadkeys(quadkeys, directory, common_params)

In [ ]:
# upload_to_osm.py #
import requests

# Define the API endpoints
osm_api_create_changeset_url = 'https://api.openstreetmap.org/api/0.6/changeset/create'
osm_api_upload_changes_url = 'https://api.openstreetmap.org/api/0.6/changeset/{}/upload'
osm_api_close_changeset_url = 'https://api.openstreetmap.org/api/0.6/changeset/{}/close'

# Set up request headers (e.g., Authorization header for authenticated requests)
headers = {
    'Content-Type': 'text/xml',
    'Authorization': 'Bearer ACCESS_TOKEN'  # Only for authenticated requests
}

# Define the XML data for creating a changeset
changeset_xml = '<osm><changeset><tag k="created_by" v="YourAppName"/></changeset></osm>'

# Send a POST request to create a changeset
create_changeset_response = requests.put(osm_api_create_changeset_url, headers=headers, data=changeset_xml)

# Check if the changeset creation was successful (status code 200)
if create_changeset_response.status_code == 200:
    changeset_id = create_changeset_response.text
    print(f'Changeset created with ID: {changeset_id}')

    # Define the XML data for adding a new node
    node_xml = '<osm><node changeset="{}" lat="LATITUDE" lon="LONGITUDE"><tag k="name" v="NewNode"/></node></osm>'.format(changeset_id)

    # Send a POST request to upload the changes
    upload_changes_response = requests.post(osm_api_upload_changes_url.format(changeset_id), headers=headers, data=node_xml)

    # Check if the changes upload was successful (status code 200)
    if upload_changes_response.status_code == 200:
        print('Changes uploaded successfully')

        # Send a PUT request to close the changeset
        close_changeset_response = requests.put(osm_api_close_changeset_url.format(changeset_id), headers=headers)

        # Check if the changeset closure was successful (status code 200)
        if close_changeset_response.status_code == 200:
            print('Changeset closed successfully')
        else:
            print(f'Error closing changeset: {close_changeset_response.status_code} - {close_changeset_response.text}')
    else:
        print(f'Error uploading changes: {upload_changes_response.status_code} - {upload_changes_response.text}')
else:
    print(f'Error creating changeset: {create_changeset_response.status_code} - {create_changeset_response.text}')


##     Replace YourAppName with the name of your application.
##     Replace ACCESS_TOKEN with your actual OSM API access token.
##     Replace LATITUDE and LONGITUDE with the coordinates of the location where you want to a